In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from collections import defaultdict
import os
import pickle
import scipy.stats as st

from itertools import combinations
from scipy.stats import chi2

from code.algorithm_alpha import algorithm_alpha
from code.algorithm_t import algorithm_t

sim = False

In [2]:
df = pd.read_csv('data/df_level2.csv')
df

,CNTSCHID,CNT,PRIVATESCH,STRATIO,SCHSIZE,sum_MATHbelow,mean_ESCS,mean_ESCS_std,Y_MATH,Y_BIN_MATH
0,800004,Albania,public,9.7619,205,11,-1.152958,-0.268234,5,1
1,800006,Albania,public,18.0000,315,15,-1.343747,-0.565595,5,1
2,800007,Albania,public,4.7368,45,2,-2.401967,-2.214922,4,1
3,800008,Albania,private,13.8125,221,6,-0.135900,1.316941,3,1
4,800009,Albania,public,16.9691,823,19,-0.560240,0.655569,2,0
...,...,...,...,...,...,...,...,...,...,...
12615,97500358,B-S-J-Z (China),public,8.0786,1131,0,0.828180,1.595021,0,0
12616,97500359,B-S-J-Z (China),public,11.3202,2298,0,-0.178994,0.251551,0,0
12617,97500360,B-S-J-Z (China),public,9.7932,2605,0,0.190169,0.743977,0,0
12618,97500361,B-S-J-Z (China),public,11.7941,2005,3,-0.929946,-0.750143,0,0


In [3]:
df['STRATIO_scaled'] = preprocessing.scale(df.STRATIO)
df['SCHSIZE_scaled'] = preprocessing.scale(df.SCHSIZE)
df['mean_ESCS_std_scaled'] = preprocessing.scale(df.mean_ESCS_std)

In [4]:
#df.CNT.unique()
df.groupby('CNT').count().reset_index()['CNT']
#df['Y_MATH'].hist()

0                  Albania
1                Argentina
2                Australia
3          B-S-J-Z (China)
4        Baku (Azerbaijan)
5                   Brazil
6        Brunei Darussalam
7                    Chile
8           Chinese Taipei
9                 Colombia
10              Costa Rica
11                 Denmark
12      Dominican Republic
13                 Estonia
14                  France
15                 Georgia
16                  Greece
17               Indonesia
18                 Ireland
19                  Israel
20                   Italy
21                   Japan
22                  Jordan
23              Kazakhstan
24                 Lebanon
25               Lithuania
26              Luxembourg
27                Malaysia
28                   Malta
29                  Mexico
30                 Moldova
31                 Morocco
32                  Panama
33                    Peru
34             Philippines
35                  Poland
36                Portugal
3

In [ ]:
ran_var = False
ran_int = True
n_fix = 1
tol = 0.01  # set as 0.01, 0.05 or 0.10 if algorithm_alpha is used; 
            # otherwise use algorithm_t and tune the tolerance tol
model = 'B' # 'B' for Bernoulli and 'P' for Poisson

variab = 'SE'

In [ ]:
N = df.CNT.nunique()

lengths = np.array(df.groupby('CNT').count().reset_index().iloc[:,1])

if model == 'B':
    y = list(df.groupby('CNT')['Y_BIN_MATH'].apply(np.array).values)
elif model == 'P':
    y = list(df.groupby('CNT')['Y_MATH'].apply(np.array).values)

fix = defaultdict(list)

if variab == "SE":
    fix[0] = df.groupby('CNT')['SCHSIZE_scaled'].apply(np.array).values.tolist() #S
    fix[1] = df.groupby('CNT')['mean_ESCS_std_scaled'].apply(np.array).values.tolist() #E
elif variab == "S":
    fix[0] = df.groupby('CNT')['SCHSIZE_scaled'].apply(np.array).values.tolist() #S
elif variab == "E":
    fix[0] = df.groupby('CNT')['mean_ESCS_std_scaled'].apply(np.array).values.tolist() #E

In [ ]:
# with algorithm_alpha

knots, par, W, hess_ran, hess_fix = algorithm_alpha(ran_var, ran_int, n_fix, sim, tol, model, fix, lengths, y, N)

In [ ]:
# with algorithm_t

# knots, par, W, hess_ran, hess_fix = algorithm_t(ran_var, ran_int, n_fix, sim, tol, model, fix, lengths, y, N)

In [ ]:
group = np.array([np.nan if np.sum(W[i,:])==0 else np.argmax(W[i,:]) for i in range(N)])

df.groupby('CNT').count().reset_index()['CNT']

In [ ]:
def loglikelihood(c, par, group, model):
        s = []  # s <- rep(0,N)
        # param_fixed = par
        # param_random = knots = c

        for i in range(N):

            if group[i]==group[i]:
              a = y[i]

              b = c[int(group[i])]
              for k in range(n_fix):
                b = b + par[k] * fix[k][i]

              if model=='B':
                s.append(np.sum(a * b - np.log(1 + np.exp(b.astype(float) ) ) )) # HO CAMBIATO QUI: ho messo .astype(float)
              elif model=='P':
                s.append(np.sum(a * b - np.exp(b.astype(float)) - np.log(np.nan_to_num(factorial(a))) ))

        return np.sum(np.array(s))

a = loglikelihood(knots, par, group, model)

In [ ]:
file_name = str(ran_var) + '_' + str(ran_int) + '_' + str(n_fix) + '_' + str(variab) + '_' + str(tol) + '_' + str(model) + '_merging' + '.pickle'
f = open('output/case_study_results' + file_name, 'wb')
#pickle.dump([par, knots, group, a, W, hess_inv_ran, hess_inv_fix],f)
pickle.dump([knots, par, W, hess_ran, hess_fix, group, a],f)
f.close()

In [ ]:
CI_length = np.array(st.norm.ppf(1 - tol / 2) * np.sqrt(1 / hess_ran))
pd.DataFrame({'lb': knots - CI_length.flatten(), 'ub': knots + CI_length.flatten()})